In [8]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# Load the iris dataset
iris = datasets.load_iris(return_X_y=True)
X, y = iris
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    "solver": "lbfgs",
    "max_iter": 200,
    "multi_class": "auto",
    "random_state": 8888,
    "penalty": "l2",
}

# Create a Logistic Regression model
model = LogisticRegression(**params)
# Train the model
model.fit(X_train, y_train)
# Make predictions
predictions = model.predict(X_test)

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [10]:

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Model accuracy: {accuracy:.2f}")

Model accuracy: 1.00


In [11]:
# Log the model with MLflow
signature = infer_signature(X_train, predictions)
with mlflow.start_run() as run:
    mlflow.log_params(params)
    mlflow.log_metric("accuracy", accuracy)
    # Set tags for the run
    mlflow.set_tag("Training info", "Basic LR model for iris dataset")
    # Log the model with signature
    infer_signature(X_train, model.predict(X_train))
    # Log the model
    model_info = mlflow.sklearn.log_model(sk_model=model, artifact_path="iris_model", signature=signature, input_example=X_train, registered_model_name = "tracking-quickstart")
    print(f"Model saved in run {run.info.run_id}")

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/04/26 11:46:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 2


Model saved in run 3475819260c849f2ae0dc4e29ea73e55
🏃 View run adorable-flea-569 at: http://127.0.0.1:5000/#/experiments/0/runs/3475819260c849f2ae0dc4e29ea73e55
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '2' of model 'tracking-quickstart'.


In [13]:
params = {
    "solver": "newton-cg",
    "max_iter": 200,
    "multi_class": "auto",
    "random_state": 1000,
}

# Create a Logistic Regression model
model = LogisticRegression(**params)
# Train the model
model.fit(X_train, y_train)

/opt/anaconda3/envs/mlops/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=200, multi_class='auto', random_state=1000,
                   solver='newton-cg')

In [14]:
# Make predictions
predictions = model.predict(X_test)
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Model accuracy: {accuracy:.2f}")

Model accuracy: 1.00


In [15]:
# Log the model with MLflow
signature = infer_signature(X_train, predictions)
with mlflow.start_run() as run:
    mlflow.log_params(params)
    mlflow.log_metric("accuracy", accuracy)
    # Set tags for the run
    mlflow.set_tag("Training info", "Basic LR model for iris dataset")
    # Log the model with signature
    infer_signature(X_train, model.predict(X_train))
    # Log the model
    model_info = mlflow.sklearn.log_model(sk_model=model, artifact_path="iris_model", signature=signature, input_example=X_train, registered_model_name = "tracking-quickstart")
    print(f"Model saved in run {run.info.run_id}")

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/04/26 11:51:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 3


Model saved in run 30866a5710e443b19297c81c7f2c897e
🏃 View run beautiful-goose-736 at: http://127.0.0.1:5000/#/experiments/0/runs/30866a5710e443b19297c81c7f2c897e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '3' of model 'tracking-quickstart'.


Inferencing and validation

In [21]:
import mlflow
from mlflow.models import Model

model_uri = 'runs:/30866a5710e443b19297c81c7f2c897e/iris_model'
pyfunc_model = mlflow.pyfunc.load_model(model_uri)
predictions = pyfunc_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names
results = pd.DataFrame(
    data=X_test,
    columns=iris_feature_names,
)
results["actual_class"] = y_test
results["predicted_class"] = predictions
results[:5]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
4,6.8,2.8,4.8,1.4,1,1
